# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Read dataframe with 16797607 records.
Dataframe has 71828 records between (datetime.datetime(2021, 2, 3, 0, 0), datetime.datetime(2021, 2, 3, 0, 0)).
Subsetted dataframe with 25000 records.
Record num : 25000
Wall time: 15.5 s


In [3]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
        
    # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
        # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)
        continue

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
    data = get_ohlc(data)    
    data = get_trend_fwd(data)
    data = trend_angle_check(data)    
    
    data = get_position(data)
    if not data['position']:
        continue
        
    # Get Dirs --------------------------------
    if len(data['dir_list']) < 2:
        data['dir_list'].append(data['position'])   
        continue

    elif len(data['dir_list']) == 2:
        data = get_cross_dir(data)
    # ----------------------------------------------------------  

 50%|█████████████████▋                 | 12607/25000 [00:44<00:43, 284.21it/s]

12591
RangeIndex(start=1792, stop=12592, step=1)
10800
RangeIndex(start=1792, stop=12592, step=1)
Int64Index([1793, 3207, 5015, 6778, 7456, 10066], dtype='int64')
[1.20453  1.20502  1.204805 1.204805 1.204805 1.20458 ]
-9.688748915774884e-09 1.204812911569841
1.2046909205322425
12592
RangeIndex(start=1793, stop=12593, step=1)
10800
RangeIndex(start=1793, stop=12593, step=1)
Int64Index([1793, 3207, 5015, 6778, 7456, 10066], dtype='int64')
[1.20453  1.20502  1.204805 1.204805 1.204805 1.20458 ]
-9.688748915774884e-09 1.204812911569841
1.2046909108434936
12593
RangeIndex(start=1794, stop=12594, step=1)
10800
RangeIndex(start=1794, stop=12594, step=1)
Int64Index([3207, 5015, 6778, 7456, 10066], dtype='int64')
[1.20502  1.204805 1.204805 1.204805 1.20458 ]
-5.6446022352384515e-08 1.2051701475077887
1.2044593227483051
12594
RangeIndex(start=1795, stop=12595, step=1)
10800
RangeIndex(start=1795, stop=12595, step=1)
Int64Index([3207, 5015, 6778, 7456, 10066], dtype='int64')
[1.20502  1.204805 

SystemExit: 

data['line_length']

(data['i'] - 1599) % data['candle_size']

data['i'] % data['candle_size']

In [ ]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

In [ ]:
plot_graph(data)

In [ ]:
data['df'].to_csv('data/temp.csv')

In [ ]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

In [ ]:
send_telegram_message(f'Run Complete at : {dt.datetime.now()}')